In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
%matplotlib inline

import os
import matplotlib.pyplot as plt

base_dir = '/kaggle/input/apple-disease-dataset/datasets'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
# image_size = (224, 224) # efficienetB0, ResNet50V2, DenseNet121, MobileNetV2
# image_size = (240, 240) # efficienetB1
# image_size = (260, 260) # efficienetB2
image_size = (300, 300) # efficienetB3

In [ ]:
# input_shapes = (224, 224, 3) # efficienetB0, ResNet50V2, DenseNet121, MobileNetV2
# input_shapes = (240, 240, 3) # efficienetB1
# input_shapes = (260, 260, 3) # efficienetB2
input_shapes = (300, 300, 3) # efficienetB3

In [ ]:
batch_size = 32
LEARNING_RATE=0.0001
num_classes=4

In [ ]:
epochs = 100

In [ ]:
trdata=tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False, samplewise_center=False,
    featurewise_std_normalization=False, samplewise_std_normalization=False,
    rotation_range=0, width_shift_range=0.0, height_shift_range=0.0,
    brightness_range=None, shear_range=0.0, zoom_range=0.0, channel_shift_range=0.0,
    horizontal_flip=False, vertical_flip=False,
    validation_split=0.2, rescale=1./255
)

train_ds = trdata.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    class_mode="categorical",
    subset="training",
    batch_size=batch_size,
    shuffle=True,
)

val_ds = trdata.flow_from_directory(
    directory=train_dir,
    target_size=image_size,
    class_mode="categorical",
    subset="validation",
    batch_size=batch_size,
    shuffle=True,
)

testdata=tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False, samplewise_center=False,
    featurewise_std_normalization=False, samplewise_std_normalization=False,
    rotation_range=0, width_shift_range=0.0, height_shift_range=0.0,
    brightness_range=None, shear_range=0.0, zoom_range=0.0, channel_shift_range=0.0,
    horizontal_flip=False, vertical_flip=False, rescale=1./255
)

test_ds = testdata.flow_from_directory(
    directory=test_dir,
    target_size=image_size,
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=False,
)

In [ ]:
x_batch, y_batch = next(train_ds)

_, axs = plt.subplots(4, 8, figsize=(20, 10))
axs = axs.flatten()
for i, ax in zip(range(0, 32), axs):
    image = x_batch[i]
    ax.axis("off")
    ax.imshow(image)

plt.savefig("preprocessed.png")
plt.show()

In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet

In [ ]:
from keras import layers
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.applications import MobileNetV2

from keras import optimizers

# base_model = EfficientNetB0(include_top=False, input_shape=input_shapes, pooling='avg', weights="imagenet",classes=num_classes)
# base_model = EfficientNetB1(include_top=False, input_shape=input_shapes, pooling='avg', weights="imagenet",classes=num_classes)
# base_model = EfficientNetB2(include_top=False, input_shape=input_shapes, pooling='avg', weights="imagenet",classes=num_classes)
base_model = EfficientNetB3(include_top=False, input_shape=input_shapes, pooling='avg', weights="imagenet",classes=num_classes)
# base_model = ResNet50V2(include_top=False, input_shape=input_shapes, pooling='avg', weights="imagenet",classes=num_classes)
# base_model = DenseNet121(include_top=False, input_shape=input_shapes, pooling='avg', weights="imagenet",classes=num_classes)
# base_model = MobileNetV2(include_top=False, input_shape=input_shapes, pooling='avg', weights="imagenet",classes=num_classes)

x = base_model.output   

x = BatchNormalization()(x)

# Start of Neural Network Part
x = Dense(512,activation='sigmoid')(x)    # 128/256/512/1024
x = BatchNormalization()(x)
#x = Activation(swish_act)(x)
x = Dropout(0.5)(x)  # 0.1/0.2/0.3/0.5/0.6/0.7

x = Dense(128,activation='sigmoid')(x)   # 128/256/512/1024
x = BatchNormalization()(x)
#x = Activation(swish_act)(x)
x = Dropout(0.7)(x)  # 0.1/0.2/0.3/0.5/0.6/0.7

predictions = Dense(num_classes, activation="softmax")(x)

model= Model(inputs = base_model.input, outputs = predictions)

model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=LEARNING_RATE),metrics=['acc'])
model.summary()
model.save('model_base.h5')

In [ ]:
from keras.callbacks import ModelCheckpoint,EarlyStopping

checkpoint = ModelCheckpoint('effB3.h5', monitor='val_acc',save_best_only=True,)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=20, verbose=1, mode='auto')
history=model.fit(train_ds, epochs=epochs, callbacks=[checkpoint,early], validation_data=val_ds)

In [ ]:
import matplotlib.pyplot as plt
import sklearn
from itertools import product
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from keras.models import load_model

model.load_weights('effB3.h5')

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes, rotation=0)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range (cm.shape[0]):
        for j in range (cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig('cnf_B3.jpg')

class_labels = test_ds.class_indices
class_labels = {v: k for k, v in class_labels.items()}
classes = list(class_labels.values())


Y_pred = model.predict_generator(test_ds)
y_pred = np.argmax(Y_pred, axis=1)

target_names = list(class_labels.values())

predicted_classes = np.argmax(Y_pred, axis=1)

cnf_matrix = confusion_matrix(test_ds.classes, y_pred)
plt.figure(figsize=(10, 10))
plt.rcParams.update({'font.size': 16})

plot_confusion_matrix(cnf_matrix, classes=classes,
                      title='')
plt.show()

print(classification_report(test_ds.classes, y_pred, target_names=target_names,digits=4))